# Exercises

## Exercise 3.1: Structuring a program as a collection of functions

``` python
# report.py
import csv

def read_portfolio(filename):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    portfolio = []
    with open(filename) as f:
        rows = csv.reader(f)
        headers = next(rows)

        for row in rows:
            record = dict(zip(headers, row))
            stock = {
                'name' : record['name'],
                'shares' : int(record['shares']),
                'price' : float(record['price'])
            }
            portfolio.append(stock)
    return portfolio
...
```


Modify your report.py program so that all major operations, including calculations and output, are carried out by a collection of functions. Specifically:

- Create a function print_report(report) that prints out the report.
- Change the last part of the program so that it is nothing more than a series of function calls and no other computation.

Original operation:

``` python
headers = ('Name', 'Shares', 'Price', 'Change')
print('%10s %10s %10s %10s'  % headers)
print(('-' * 10 + ' ') * len(headers))

for row in report:
    print('%10s %10d %10.2f %10.2f' % row)
```

Function option:

``` python
def print_report(reportdata):
    '''
    Print a nicely formated table from a list of (name, shares, price, change) tuples.
    '''
    headers = ('Name','Shares','Price','Change')
    print('%10s %10s %10s %10s' % headers)
    print(('-'*10 + ' ')*len(headers))
    for row in reportdata:
        print('%10s %10d %10.2f %10.2f' % row)
```

## Exercise 3.2: Creating a top-level function for program execution

In [14]:
# report.py
import csv

def read_portfolio(filename):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    portfolio = []
    with open(filename) as f:
        rows = csv.reader(f)
        headers = next(rows)

        for row in rows:
            record = dict(zip(headers, row))
            stock = {
                'name' : record['name'],
                'shares' : int(record['shares']),
                'price' : float(record['price'])
            }
            portfolio.append(stock)

    return portfolio

def read_prices(filename):
    '''
    Read a CSV file of price data into a dict mapping names to prices.
    '''
    prices = {}
    with open(filename) as f:
        rows = csv.reader(f)
        for row in rows:
            try:
                prices[row[0]] = float(row[1])
            except IndexError:
                pass

    return prices

def make_report_data(portfolio,prices):
    '''
    Make a list of (name, shares, price, change) tuples given a portfolio list
    and prices dictionary.
    '''
    rows = []
    for stock in portfolio:
        current_price = prices[stock['name']]
        change = current_price - stock['price']
        summary = (stock['name'], stock['shares'], current_price, change)
        rows.append(summary)
    return rows

def print_report(reportdata):
    '''
    Print a nicely formated table from a list of (name, shares, price, change) tuples.
    '''
    headers = ('Name','Shares','Price','Change')
    print('%10s %10s %10s %10s' % headers)
    print(('-'*10 + ' ')*len(headers))
    for row in reportdata:
        print('%10s %10d %10.2f %10.2f' % row)

def portfolio_report(portfoliofile,pricefile):        
    '''
    Make a stock report given portfolio and price data files.
    '''
    # Read data files 
    portfolio = read_portfolio(portfoliofile)
    prices = read_prices(pricefile)

    # Create the report data
    report = make_report_data(portfolio,prices)

    # Print it out
    print_report(report)
    
portfolio_report('Data/portfolio.csv', 'Data/prices.csv')

      Name     Shares      Price     Change
---------- ---------- ---------- ---------- 
        AA        100       9.22     -22.98
       IBM         50     106.28      15.18
       CAT        150      35.46     -47.98
      MSFT        200      20.89     -30.34
        GE         95      13.48     -26.89
      MSFT         50      20.89     -44.21
       IBM        100     106.28      35.84


## Exercise 3.3: Reading CSV Files

Define this function:

In [15]:
# fileparse.py
import csv

def parse_csv(filename):
    '''
    Parse a CSV file into a list of records
    '''
    with open(filename) as f:
        rows = csv.reader(f)

        # Read the file headers
        headers = next(rows)
        records = []
        for row in rows:
            if not row:    # Skip rows with no data
                continue
            record = dict(zip(headers, row))
            records.append(record)

    return records

In [16]:
portfolio = parse_csv('Data/portfolio.csv')
portfolio

# This function reads a CSV file into a list of dictionaries while
# hiding the details of opening the file, wrapping it with the csv module,
# ignoring blank lines, and so forth.

[{'name': 'AA', 'shares': '100', 'price': '32.20'},
 {'name': 'IBM', 'shares': '50', 'price': '91.10'},
 {'name': 'CAT', 'shares': '150', 'price': '83.44'},
 {'name': 'MSFT', 'shares': '200', 'price': '51.23'},
 {'name': 'GE', 'shares': '95', 'price': '40.37'},
 {'name': 'MSFT', 'shares': '50', 'price': '65.10'},
 {'name': 'IBM', 'shares': '100', 'price': '70.44'}]

## Exercise 3.4: Building a Column Selector
Modify the parse_csv() function so that it optionally allows user-specified columns to be picked out.

``` python
        if select:
            indices = [headers.index(colname) for colname in select]
            headers = select
        else:
            indices = []
```

In [17]:
def parse_csv(filename, select=None):
    '''
    Parse a CSV file into a list of records
    '''
    with open(filename) as f:
        rows = csv.reader(f)

        # Read the file headers
        headers = next(rows)

        # If a column selector was given, find indices of the specified columns.
        # This maps the column selections to row indices.
        # Also narrow the set of headers used for resulting dictionaries.
        if select:
            indices = [headers.index(colname) for colname in select]
            headers = select
        else:
            indices = []

        records = []
        for row in rows:
            if not row:    # Skip rows with no data
                continue
            # Filter the row if specific columns were selected
            if indices:
                row = [ row[index] for index in indices ]

            # Make a dictionary
            record = dict(zip(headers, row))
            records.append(record)

    return records

In [18]:
shares_held = parse_csv('Data/portfolio.csv', select=['name','shares'])
shares_held

[{'name': 'AA', 'shares': '100'},
 {'name': 'IBM', 'shares': '50'},
 {'name': 'CAT', 'shares': '150'},
 {'name': 'MSFT', 'shares': '200'},
 {'name': 'GE', 'shares': '95'},
 {'name': 'MSFT', 'shares': '50'},
 {'name': 'IBM', 'shares': '100'}]

## Exercise 3.5: Performing Type Conversion
Modify the parse_csv() function so that it optionally allows type-conversions to be applied to the returned data.

``` python
if types:
    row = [func(val) for func, val in zip(types, row) ]
```

In [19]:
def parse_csv(filename, select=None, types=None):
    '''
    Parse a CSV file into a list of records
    '''
    with open(filename) as f:
        rows = csv.reader(f)

        # Read the file headers
        headers = next(rows)

        # If a column selector was given, find indices of the specified columns.
        # Also narrow the set of headers used for resulting dictionaries
        if select:
            indices = [headers.index(colname) for colname in select]
            headers = select
        else:
            indices = []

        records = []
        for row in rows:
            if not row:    # Skip rows with no data
                continue
            
            # Filter the row if specific columns were selected
            if indices:
                row = [ row[index] for index in indices ]
            
            # func is an argument given to the function `identity_decorator()`
            # zip an iterator of tuples where the items in each passed iterator are paired together
            if types:
                row = [func(val) for func, val in zip(types, row) ]

            # Make a dictionary
            record = dict(zip(headers, row))
            records.append(record)

    return records

In [20]:
portfolio = parse_csv('Data/portfolio.csv', types=[str, int, float])
portfolio

[{'name': 'AA', 'shares': 100, 'price': 32.2},
 {'name': 'IBM', 'shares': 50, 'price': 91.1},
 {'name': 'CAT', 'shares': 150, 'price': 83.44},
 {'name': 'MSFT', 'shares': 200, 'price': 51.23},
 {'name': 'GE', 'shares': 95, 'price': 40.37},
 {'name': 'MSFT', 'shares': 50, 'price': 65.1},
 {'name': 'IBM', 'shares': 100, 'price': 70.44}]

## Exercise 3.6: Working without Headers
Modify the parse_csv() function so that it can work with such files by creating a list of tuples instead. 

``` python
    if headers:
        record = dict(zip(headers, row))
    else:
        record = tuple(row)
        records.append(record)
```

In [21]:
# fileparse.py
import csv

def parse_csv(filename, select=None, types=None, has_headers=True, delimiter=','):
    '''
    Parse a CSV file into a list of records with type conversion.
    '''
    with open(filename) as f:
        rows = csv.reader(f, delimiter=delimiter)

        # If has headers, use next(rows), otherwise blank
        # The next() function returns the next item in an iterator.
        if has_headers:
            headers = next(rows)
        else:
            headers = []

        # If specific columns have been selected, make indices for filtering 
        if select:
            indices = [ headers.index(colname) for colname in select ]
            headers = select

        records = []
        for row in rows:
            if not row:     # Skip rows with no data
                continue

            # If specific column indices are selected, pick them out
            if select:
                row = [ row[index] for index in indices]

            # Apply type conversion to the row
            if types:
                row = [func(val) for func, val in zip(types, row)]

            # Make a dictionary or a tuple
            if headers:
                record = dict(zip(headers, row))
            else:
                record = tuple(row)
            records.append(record)

        return records

In [22]:
prices = parse_csv('Data/prices.csv', types=[str,float], has_headers=False)
prices

[('AA', 9.22),
 ('AXP', 24.85),
 ('BA', 44.85),
 ('BAC', 11.27),
 ('C', 3.72),
 ('CAT', 35.46),
 ('CVX', 66.67),
 ('DD', 28.47),
 ('DIS', 24.22),
 ('GE', 13.48),
 ('GM', 0.75),
 ('HD', 23.16),
 ('HPQ', 34.35),
 ('IBM', 106.28),
 ('INTC', 15.72),
 ('JNJ', 55.16),
 ('JPM', 36.9),
 ('KFT', 26.11),
 ('KO', 49.16),
 ('MCD', 58.99),
 ('MMM', 57.1),
 ('MRK', 27.58),
 ('MSFT', 20.89),
 ('PFE', 15.19),
 ('PG', 51.94),
 ('T', 24.79),
 ('UTX', 52.61),
 ('VZ', 29.26),
 ('WMT', 49.74),
 ('XOM', 69.35)]

## Exercise 3.7: Picking a different column delimitier
Modify your parse_csv() function so that it also allows the delimiter to be changed.

``` python
delimiter=','
```

In [23]:
# fileparse.py
import csv

def parse_csv(filename, select=None, types=None, has_headers=True, delimiter=','):
    '''
    Parse a CSV file into a list of records with type conversion.
    '''
    with open(filename) as f:
        rows = csv.reader(f, delimiter=delimiter)

        # If has headers, use next(rows), otherwise blank
        if has_headers:
            headers = next(rows)
        else:
            headers = []

        # If specific columns have been selected, make indices for filtering 
        if select:
            indices = [ headers.index(colname) for colname in select ]
            headers = select

        records = []
        for row in rows:
            if not row:     # Skip rows with no data
                continue

            # If specific column indices are selected, pick them out
            if select:
                row = [ row[index] for index in indices]

            # Apply type conversion to the row
            if types:
                row = [func(val) for func, val in zip(types, row)]

            # Make a dictionary or a tuple
            if headers:
                record = dict(zip(headers, row))
            else:
                record = tuple(row)
            records.append(record)

        return records

## Exercise 3.8: Raising exceptions

Modify the code so that an exception gets raised if both the select and has_headers=False arguments are passed.

``` python
    if select and not has_headers:
        raise RuntimeError('select requires column headers')
```

In [24]:
# fileparse.py
import csv

def parse_csv(filename, select=None, types=None, has_headers=True, delimiter=','):
    '''
    Parse a CSV file into a list of records with type conversion.
    '''
    if select and not has_headers:
        raise RuntimeError('select requires column headers')
        
    with open(filename) as f:
        rows = csv.reader(f, delimiter=delimiter)

        # If has headers, use next(rows), otherwise blank
        if has_headers:
            headers = next(rows)
        else:
            headers = []

        # If specific columns have been selected, make indices for filtering 
        if select:
            indices = [ headers.index(colname) for colname in select ]
            headers = select

        records = []
        for row in rows:
            if not row:     # Skip rows with no data
                continue

            # If specific column indices are selected, pick them out
            if select:
                row = [ row[index] for index in indices]

            # Apply type conversion to the row
            if types:
                row = [func(val) for func, val in zip(types, row)]

            # Make a dictionary or a tuple
            if headers:
                record = dict(zip(headers, row))
            else:
                record = tuple(row)
            records.append(record)

        return records

## Exercise 3.9: Catching exceptions

Modify the parse_csv() function to catch all ValueError exceptions generated during record creation and print a warning message for rows that can’t be converted.

``` python
if types:
    try:
        row = [func(val) for func, val in zip(types, row)]
    except ValueError as e:
        print(f"Row {rowno}: Couldn't convert {row}")
        print(f"Row {rowno}: Reason {e}")
    continue
```

In [26]:
# fileparse.py
import csv

def parse_csv(filename, select=None, types=None, has_headers=True, delimiter=','):
    '''
    Parse a CSV file into a list of records with type conversion.
    '''
    if select and not has_headers:
        raise RuntimeError('select requires column headers')
        
    with open(filename) as f:
        rows = csv.reader(f, delimiter=delimiter)

        # If has headers, use next(rows), otherwise blank
        if has_headers:
            headers = next(rows)
        else:
            headers = []

        # If specific columns have been selected, make indices for filtering 
        if select:
            indices = [ headers.index(colname) for colname in select ]
            headers = select

        records = []
        for row in rows:
            if not row:     # Skip rows with no data
                continue

            # If specific column indices are selected, pick them out
            if select:
                row = [ row[index] for index in indices]

            # Apply type conversion to the row
            if types:
                row = [func(val) for func, val in zip(types, row)]
        
            if types:
                try:
                    row = [func(val) for func, val in zip(types, row)]
                except ValueError as e:
                        print(f"Row {rowno}: Couldn't convert {row}")
                        print(f"Row {rowno}: Reason {e}")
                    continue

            # Make a dictionary or a tuple
            if headers:
                record = dict(zip(headers, row))
            else:
                record = tuple(row)
            records.append(record)

        return records

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 44)

## Exercise 3.10: Silencing Errors

Modify the parse_csv() function so that parsing error messages can be silenced if explicitly desired by the user. 

``` python
        if types:
                try:
                    row = [func(val) for func, val in zip(types, row)]
                except ValueError as e:
                    if not silence_errors:
                        print(f"Row {rowno}: Couldn't convert {row}")
                        print(f"Row {rowno}: Reason {e}")
                    continue
```

In [ ]:
# fileparse.py
import csv

def parse_csv(filename, select=None, types=None, has_headers=True, delimiter=',', silence_errors=False):
    '''
    Parse a CSV file into a list of records with type conversion.
    '''
    if select and not has_headers:
        raise RuntimeError('select requires column headers')
        
    with open(filename) as f:
        rows = csv.reader(f, delimiter=delimiter)

        # If has headers, use next(rows), otherwise blank
        if has_headers:
            headers = next(rows)
        else:
            headers = []

        # If specific columns have been selected, make indices for filtering 
        if select:
            indices = [ headers.index(colname) for colname in select ]
            headers = select

        records = []
        for row in rows:
            if not row:     # Skip rows with no data
                continue

            # If specific column indices are selected, pick them out
            if select:
                row = [ row[index] for index in indices]

            # Apply type conversion to the row
            if types:
                row = [func(val) for func, val in zip(types, row)]
        
            if types:
                try:
                    row = [func(val) for func, val in zip(types, row)]
                except ValueError as e:
                    if not silence_errors:
                        print(f"Row {rowno}: Couldn't convert {row}")
                        print(f"Row {rowno}: Reason {e}")
                    continue

            # Make a dictionary or a tuple
            if headers:
                record = dict(zip(headers, row))
            else:
                record = tuple(row)
            records.append(record)

        return records

## Exercise 3.11: Module imports